In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sohier/vesuvius-metric-resources")

print("Path to dataset files:", path)

100%|██████████| 133M/133M [00:15<00:00, 9.14MB/s] 

Extracting files...


Path to dataset files: C:\Users\user\.cache\kagglehub\datasets\sohier\vesuvius-metric-resources\versions\15


# Metric Test

In [2]:
import os
from topometrics import compute_leaderboard_score
import tifffile
import glob
import kagglehub

data_path = kagglehub.competition_download('vesuvius-challenge-surface-detection')
print("Data Load Complete")
test_data_dir = os.path.join(data_path, "test_images")
train_data_dir = os.path.join(data_path, "train_images")
train_lable_dir = os.path.join(data_path, "train_labels")

train_data = sorted(glob.glob(os.path.join(train_data_dir, "*")))
sample = train_data[:10]
label = sorted(glob.glob(os.path.join(train_lable_dir, "*")))

print(label)

# pr, gt are 3D arrays with identical shape (Z, Y, X)
pr = tifffile.imread(label[0])  # Here we just use the ground truth as a dummy prediction   
gt = tifffile.imread(label[0])
print("Prediction shape:", pr.shape)
print("Ground truth shape:", gt.shape)




/home/user/miniconda3/envs/vesuvius/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Data Load Complete
['/home/user/.cache/kagglehub/competitions/vesuvius-challenge-surface-detection/train_labels/1004283650.tif', '/home/user/.cache/kagglehub/competitions/vesuvius-challenge-surface-detection/train_labels/1006462223.tif', '/home/user/.cache/kagglehub/competitions/vesuvius-challenge-surface-detection/train_labels/1013184726.tif', '/home/user/.cache/kagglehub/competitions/vesuvius-challenge-surface-detection/train_labels/102536988.tif', '/home/user/.cache/kagglehub/competitions/vesuvius-challenge-surface-detection/train_labels/1029212680.tif', '/home/user/.cache/kagglehub/competitions/vesuvius-challenge-surface-detection/train_labels/1033375083.tif', '/home/user/.cache/kagglehub/competitions/vesuvius-challenge-surface-detection/train_labels/1033784946.tif', '/home/user/.cache/kagglehub/competitions/vesuvius-challenge-surface-detection/train_labels/1044587645.tif', '/home/user/.cache/kagglehub/competitions/vesuvius-challenge-surface-detection/train_labels/105068588.tif', '

In [ ]:
print("Computing leaderboard score...")
rep = compute_leaderboard_score(
    predictions=pr,
    labels=gt,
    dims=(0,1,2),
    spacing=(1.0, 1.0, 1.0),          # (z, y, x)
    surface_tolerance=2.0,            # in spacing units
    voi_connectivity=26,
    voi_transform="one_over_one_plus",
    voi_alpha=0.3,
    combine_weights=(0.3, 0.35, 0.35),  # (Topo, SurfaceDice, VOI)
    fg_threshold=None,                # None => legacy "!= 0"; else uses "x > threshold"
    ignore_label=2,                   # voxels with this GT label are ignored
    ignore_mask=None,                 # or pass an explicit boolean mask
)

print("Leaderboard score:", rep.score)                # scalar in [0,1]
print("Topo score:", rep.topo.toposcore)              # [0,1]
print("Surface Dice:", rep.surface_dice)              # [0,1]
print("VOI score:", rep.voi.voi_score)                # (0,1]
print("VOI split/merge:", rep.voi.voi_split, rep.voi.voi_merge)
print("Params used:", rep.params)

# Visual

In [5]:
from monai.inferers import sliding_window_inference

roi_size = (160, 160, 160)   # patch size
sw_batch_size = 2            # 한 번에 몇 patch를 GPU에 넣을지 (VRAM에 맞게)
overlap = 0.25               # patch 겹침 비율 (0~0.75 정도)
mode = "gaussian"

In [6]:
import torch
from monai.networks.nets import UNet
import tifffile as tiff

model = UNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=1,
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    num_res_units=2,
).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

inputs = tiff.imread(sample[0]) # 3D 이미지 데이터 (C, H, W, D) 형태여야 함
gt = tiff.imread(label[0])    # 정답 마스크 (C, H, W, D) 형태여야 함

print("Input shape:", inputs.shape)
print("Ground truth shape:", gt.shape)

inputs = inputs[None, None, ...]  # (1, 1, H, W, D)
gt = gt[None, None, ...]          # (1, 1, Hs, Ws, Ds)

print("Input shape after adding batch and channel dims:", inputs.shape)
print("Ground truth shape after adding batch and channel dims:", gt.shape)

Input shape: (320, 320, 320)
Ground truth shape: (320, 320, 320)
Input shape after adding batch and channel dims: (1, 1, 320, 320, 320)
Ground truth shape after adding batch and channel dims: (1, 1, 320, 320, 320)


In [ ]:
inputs = torch.tensor(inputs, dtype=torch.float32).to("cuda")

with torch.inference_mode():
     with torch.autocast("cuda", dtype=torch.float16):
        logits = sliding_window_inference(
            inputs=inputs,
            roi_size=roi_size,
            sw_batch_size=sw_batch_size,
            predictor=model,         # model(inputs_patch) -> logits_patch
            overlap=overlap,
            mode=mode,
        )
        
print("Logits shape:", logits.shape)  # (1, 1, D, H, W)

/tmp/ipykernel_19428/592322614.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs, dtype=torch.float32).to("cuda")


Logits shape: torch.Size([1, 1, 320, 320, 320])


In [ ]:
import napari
napari.view_image(inputs.squeeze(0).squeeze(0))  # (D,H,W)

/tmp/ipykernel_17315/2121559178.py:2: FutureWarning: `napari.view_image` is deprecated and will be removed in napari 0.7.0.
Use `viewer = napari.Viewer(); viewer.add_image(...)` instead.
  napari.view_image(inputs.squeeze(0).squeeze(0))  # (D,H,W)

Available platform plugins are: eglfs, linuxfb, minimal, minimalegl, offscreen, vnc, wayland-egl, wayland, wayland-xcomposite-egl, wayland-xcomposite-glx, webgl, xcb.



: 